In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk 
!pip install konlpy JPype1-py3 
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import re

import pandas as pd

import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab

In [ ]:
# https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=014&aid=0004711152
head = 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid='

urls=[
      f'{head}014&aid=0004711152',
    #   f'{head}018&aid=0005040968',
    #   f'{head}025&aid=0003136112',
    #   f'{head}629&aid=0000106856',
    #   f'{head}277&aid=0004971649',
    #   f'{head}011&aid=0003964240',
    #   f'{head}028&aid=0002560999',
    #   f'{head}003&aid=0010728873'
]

In [ ]:

RE_EMAIL = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+[a-zA-Z0-9-.]+')
def find_likely_email(text):
    m = RE_EMAIL.search(text)
    return m.group(0) if m else None


#url = urls[0]
for url in urls:
    headers = {"user-agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    body = soup.select_one('#articleBodyContents')
    title = soup.select_one('#articleTitle').get_text()
    byline = soup.select_one('.b_text').get_text().replace("\n", "").strip()
    print(body)
    #print(find_likely_email(byline))



<div class="_article_body_contents" id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->
<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>
<strong class="media_end_summary">與, "윤석열은 범죄의혹 투성이 후보" 규정<br/>"고발 사주 의혹 수사 관련 중대성·시급성 등 강조"</strong> <span class="end_photo_org"><img alt="" src="https://imgnews.pstatic.net/image/014/2021/09/18/0004711152_001_20210918101704821.jpg?type=w647"><em class="img_desc">윤석열 국민의힘 대선 경선 예비후보가 지난 16일 서울 중구 TV조선에서 열린 방송토론회에서 토론 준비 중 시계를 보고 있다. 사진=뉴시스</em></img></span> <br/>[파이낸셜뉴스]더불어민주당은 추석 연휴 기간 동안 국민의힘 대선주자인 윤석열 전 검찰총장에 대한 대국민 여론전에 당력을 집중키로 했다.  <br/> <br/>특히 '추석 밥상 민심' 주도권을 확보하기 위해 '윤석열은 불량 후보'라는 키워드를 구전 홍보키로 했다.  <br/> <br/>18일 파이낸셜뉴스 취재에 따르면 민주당은 이 같은 '추석 메시지 전략'을 세우고 당 지도부 등과 공유했다.  <br/> <br/>민주당 관계자는 "민주당은 윤 전 총장을 '범죄의혹 투성이 후보', '망언 투성이 후보'로 규정하기로 했다"면서 "추석 연휴 동안 윤 전 총장은 '불안한 후보', '불량 후보'라는 메시지를 구전 홍보하는 전략을 세웠다"고 밝혔다.  <br/> <br/>민주당은 진실공방 양상으로 흐르고 있는 '고발 사주 의혹' 관련 수사확대 여론전

In [ ]:

section = re.findall('<section[^>]*>\s*((?:.|\n)*?)<\/section>', body)

content = re.findall('<p[^>]*>(.*?)<\/p>', ''.join(section)) # p 태그 내용 추출
content = re.sub('<img\s(.*?)>', '', '\n'.join(content)) # img 태그 내용 제거
content = re.sub('(<span[^>]*>|<strong>|<\/span>|<\/strong>)', '', content) # strong 태그 제거

category = re.findall('\"category\":\"\s*((?:.|\n)*?)\"', news["mobileNews"])

print

TypeError: ignored

In [ ]:
byline

'송주용 기자 (juyong@fnnews.com)'

In [ ]:
DOCKEY_CONTENT_LENGTH = "content_length"
DOCKEY_BYLINES = "bylines"
DOCKEY_QUOTES = "quotes"
DOCKEY_TITLE = "title"
DOCKEY_TITLE_NUM_EXCLAMATION = "title_numExclamation"
DOCKEY_TITLE_NUM_QUESTION = "title_numQuestion"
DOCKEY_CONTENT_NUM_NUMBER = "content_numNumber"
DOCKEY_IMAGE_COUNT = "image_count"
DOCKEY_CONTENT_AVG_SENTENCE_LENGTH = "content_avgSentenceLength"
DOCKEY_TITLE_ADVERBS = "title_adverbs"
DOCKEY_CONTENT_AVG_ADVERBS_PER_SENTENCE = "content_avgAdverbsPerSentence"
DOCKEY_CONTENT_QUOTE_PERCENT = "content_quotePercent"

In [ ]:
doc = {
    DOCKEY_BYLINES : {'email':'송주용 기자'}
}

In [ ]:
doc = {
    DOCKEY_BYLINES : {'email':'송주용 기자'}
}

# 바이라인
has_email = False
has_name = False
if "bylines" in doc:
    for bl in doc[DOCKEY_BYLINES]:
        if "name" in bl:
            has_name = True
        if "email" in bl:
            has_email = True

score_byline = 0
if has_name:
    if has_email:
        score_byline = 1  # 실명+이메일
    else:
        score_byline = 0.8  # 이름만 있음
elif has_email:
    score_byline = 0  # 뭐 하나라도 있음
else:
    score_byline = -1  # 둘다 없음

score_byline

0

In [ ]:
# 바이라인 이메일 분리 테스트.
RE_EMAIL = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+[a-zA-Z0-9-.]+')
def find_likely_email(text):
    m = RE_EMAIL.search(text)
    return m.group(0) if m else None

In [ ]:
print(find_likely_email(byline))

juyong@fnnews.com


In [ ]:
RE_EMAIL = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+[a-zA-Z0-9-.]+')
RE_ALPHA_NUMERIC_IDENTIFIER = re.compile(r'[a-zA-Z0-9_.+-]+@?')
RE_JOURNALIST_JOB = re.compile(r'인턴기자|기자|특파원|논설위원|논설실장')
RE_TV_JOURNALIST = re.compile(r'(MBC|SBS|KBS|YTN).*?\s(.+)입')
RE_YTN_JOURNALIST = re.compile(r'YTN.*?\s(.+?)\[([a-zA-Z0-9-]*)')
RE_BYLINE_DIV_CHARS = re.compile(r'[·/|\[\]]')

# FIXME: 지역명은 따로 빼야 하는데.
RE_COMPANY_NAMES = re.compile(r'아시아투데이|한경닷컴|충청일보|중부매일|포항|디지털뉴스국')

S_PUNCTUATION_CHARS = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~‘’“”▶▷■▦△▲◇"""
S_WHITE_SPACES = " \t\xa0\xeb\u3000\u2028\u2029"

RE_SPACE_CHAR = re.compile(u'( |\t|\xa0|\xeb|\u3000|\u2028|\u2029)+')

# text 에서 꽤나 email 스러운 것을 찾아서 리턴합니다.
def find_likely_email(text):
    m = RE_EMAIL.search(text)
    return m.group(0) if m else None


# text 에서 알파벳과 숫자 _, . 등의 연속으로 이루어진 단어를 찾아 리턴합니다.
# 이메일 같이 생겼는데 @ 문자가 빠진 경우에 사용합니다. (아이디만 있는 경우)
def find_alpha_numeric(text):
    m = RE_ALPHA_NUMERIC_IDENTIFIER.search(text)
    return m.group(0) if m else None


# 바이라인 유추가 가능한 "기자", "특파원", "리포터" 등의 한글 직업명을 찾아서 리턴합니다.
def find_journalist_job(text):
    m = RE_JOURNALIST_JOB.search(text)
    return m.group(0) if m else None


# 주어진 텍스트가 바이라인일 가능성이 있나?
def can_be_byline(text):
    return True if find_likely_email(text) or find_journalist_job(text) else False


def split_get_left_right(long_text, split_word, nostrip=False):
    idx = long_text.rfind(split_word)
    if idx < 0:
        return (None,None)
    idx2 = idx + len(split_word)
    if nostrip:
        return (long_text[:idx], long_text[idx2:])
    return (long_text[:idx].strip(), long_text[idx2:].strip())



def head_to_tail_until_punctuation_char(text):
    if not text:
        return None
    for n, ch in enumerate(text):
        if ch in S_PUNCTUATION_CHARS:
            return text[:n].rstrip()
    return text.rstrip()


def tail_to_head_until_punctuation_char(text):
    if not text:
        return None
    for n, ch in enumerate(text[::-1]):
        if ch in S_PUNCTUATION_CHARS:
            return text[(len(text) - n):].lstrip()
    return text.lstrip()


# "홍길동기자" --> "홍길동 기자"
def normalize_name_with_job(namepart,jobname):
    if not namepart or not jobname:
        return None

    # 아시아투데이 박아람 기자 --> 박아람 기자
    company = RE_COMPANY_NAMES.search(namepart)
    if company:
        namepart = namepart.replace(company.group(0), '').strip()

    if len(namepart) > 0 and namepart[-1] in S_WHITE_SPACES:
        # 직업명 앞에 띄워쓰기가 있으므로 별도 처리가 필요 없다.
        namepart = namepart.strip()
        if not namepart:
            return None # 기자 앞에 이름이 없다.
        return (namepart + ' ' + jobname)

    num_spaces = len(RE_SPACE_CHAR.findall(namepart))
    if num_spaces > 0:
        # "홍길동 수습기자" 같은 경우 .. 새로운 직업명일 수 있다.
        return (namepart + jobname)

    # "홍길동기자" 같은 경우, 띄워쓰기를 넣어준다.
    return (namepart + ' ' + jobname)


class BylineExtractor(object):
    GENERAL = 1
    TV = 2

    def __init__(self, text, type=GENERAL):
        if type == BylineExtractor.TV:
            self.analyze_tv(text)
        else:
            self.analyze(text)


    def analyze(self, text):
        self.original_text = text
        self.components = dict()

        maybe_email = find_likely_email(text)
        if maybe_email:
            self.components["email"] = maybe_email
            left, right = split_get_left_right(text, maybe_email)

            maybe_job = find_journalist_job(left)
            if maybe_job:
                maybe_name = left
                redundant_job = find_journalist_job(right)
            else:
                redundant_job = None
                maybe_job = find_journalist_job(right)
                maybe_name = right if maybe_job else None


            if redundant_job:
                # TODO: log doubtful case
                logging.warning("redundant journalist job found in '%s'", text)

            if maybe_name:
                maybe_name, name_right = split_get_left_right(maybe_name, maybe_job, nostrip=True)
                maybe_name = tail_to_head_until_punctuation_char(maybe_name)
                name_right = head_to_tail_until_punctuation_char(name_right)
                name = normalize_name_with_job(maybe_name, maybe_job)
                if name:
                    # 기자 뒤 email 앞에 텍스트가 있을 경우 기자명에 포함시킴 (예: "증시분석 전문기자 로봇 ET")
                    if name_right and name_right.strip():
                        name = name + name_right
                    self.components["name"] = name
                    return

            # 이메일 앞에 보통 이름이 오는데 "기자" 직업명이 없다. 글자수가 4글자 이하면 이름으로 간주한다.
            name = tail_to_head_until_punctuation_char(left or right)
            if name:
                if len(name) <= 4:
                    self.components["name"] = name
                elif name[-1] == "팀":
                    # 혹시 ~팀 으로 끝나는 경우 이름으로 간주 (예: 산업경제팀)
                    self.components["name"] = name

            return

        # 이메일이 없는 경우 (예: "홍길동 기자" 또는 "홍길동 기자 hong")
        # 일단 기자 등의 직업명이 보여야 한다.
        maybe_job = find_journalist_job(text)
        if maybe_job:
            left, right = split_get_left_right(text, maybe_job, nostrip=True)
            maybe_name = tail_to_head_until_punctuation_char(left)
            name = normalize_name_with_job(maybe_name, maybe_job)
            if name:
                self.components["name"] = name
                maybe_email = find_alpha_numeric(right)
                if maybe_email:
                    # 찾은 email 후보와 직업명 사이에 다른 문자가 있으면 안됨
                    between = right[:(right.find(maybe_email))]
                    for ch in between:
                        if ch not in S_PUNCTUATION_CHARS and ch not in S_WHITE_SPACES:
                            return
                    # @ 가 없지만 email 로 인정
                    self.components["email"] = maybe_email



    def analyze_tv(self, text):
        self.components = dict()
        m1 = RE_TV_JOURNALIST.search(text)
        if m1:
            self.components["name"] = m1.group(2)
            return

        m2 = RE_YTN_JOURNALIST.search(text)
        if m2:
            self.components["name"] = m2.group(1)
            maybe_email = m2.group(2)
            if maybe_email:
                self.components["email"] = maybe_email



    def get_name(self):
        return self.components["name"] if "name" in self.components else None

    def get_email(self):
        return self.components["email"] if "email" in self.components else None

    def get_component_count(self):
        return len(self.components)

    def get_result(self):
        return self.components



# 일단은 한 라인에 2개 이상의 기자가 발견되는 경우만 처리한다.
class BylineAnalyzer(object):
    def __init__(self, text, type=BylineExtractor.GENERAL):
        self.extractors = list()
        jobs = RE_JOURNALIST_JOB.findall(text)
        if len(jobs) <= 1:
            # 바이라인은 1개만 있는 것 같은 일반적인 경우
            self.extractors.append(BylineExtractor(text, type))
            return

        # 두 기자명을 가르는 문자로 / 또는 · 등을 찾아본다.
        for i in range(1,len(jobs)):
            left = text.find(jobs[i-1]) + len(jobs[i-1])
            right = text.find(jobs[i], left)
            m = RE_BYLINE_DIV_CHARS.search(text[left:right])
            if m:
                divpos = text.find(m.group(0), left)
                byline1 = text[:divpos]
                self.extractors.append(BylineExtractor(byline1))
                text = text[divpos+1:]
            else:
                break

        self.extractors.append(BylineExtractor(text))


    def get_bylines(self):
        ret = list()
        for extr in self.extractors:
            if extr.get_component_count() > 0:
                ret.append(extr.get_result())

        return ret


In [ ]:
by=BylineExtractor('송주용 기자 (juyong@fnnews.com)')
print(by.get_name())
print(by.get_email())
print(by.get_component_count())
print(by.get_result())

송주용 기자
juyong@fnnews.com
2
{'email': 'juyong@fnnews.com', 'name': '송주용 기자'}


In [ ]:
DOCKEY_CONTENT_LENGTH = "content_length"
DOCKEY_BYLINES = "bylines"
DOCKEY_QUOTES = "quotes"
DOCKEY_TITLE = "title"
DOCKEY_TITLE_NUM_EXCLAMATION = "title_numExclamation"
DOCKEY_TITLE_NUM_QUESTION = "title_numQuestion"
DOCKEY_CONTENT_NUM_NUMBER = "content_numNumber"
DOCKEY_IMAGE_COUNT = "image_count"
DOCKEY_CONTENT_AVG_SENTENCE_LENGTH = "content_avgSentenceLength"
DOCKEY_TITLE_ADVERBS = "title_adverbs"
DOCKEY_CONTENT_AVG_ADVERBS_PER_SENTENCE = "content_avgAdverbsPerSentence"
DOCKEY_CONTENT_QUOTE_PERCENT = "content_quotePercent"

doc = {
    DOCKEY_BYLINES : by.get_result()
}

# 바이라인
has_email = False
has_name = False
if "bylines" in doc:
    for bl in doc[DOCKEY_BYLINES]:
        if "name" in bl:
            has_name = True
        if "email" in bl:
            has_email = True

score_byline = 0
if has_name:
    if has_email:
        score_byline = 1  # 실명+이메일
    else:
        score_byline = 0.8  # 이름만 있음
elif has_email:
    score_byline = 0  # 뭐 하나라도 있음
else:
    score_byline = -1  # 둘다 없음

score_byline

1

# 예시기사

In [ ]:
title = "코로나19, 악의적인 '교란정보'가 가장 큰 문제"

body = """
감염병은 잘못된 정보가 바이러스처럼 퍼지는 것이 실제 바이러스 확산만큼 문제다. 
이는 한국만의 문제도 아니다. 
'미디어와 코로나 판데믹(pandemic, 전염병의 지구적 유행)'을 주제로 한국언론진흥재단(이사장 민병욱)이 9일 개최한 한미 언론 합동 토론회 에서는 '치료적 보도'로서 언론의 역할에 대한 주문과 함께 다양한 제언이 등장했다. 
코로나19 국면에서 언론의 점수는 좋지 않다. KBS 공영미디어연구소가 3월 31일~4 월 2일까지 성인 남녀 1,069명을 대상으로 진행한 설문 결과 주요 대응 주체별 평가에서 질병관리본부(91.6%)가 가장 잘 대응하고 있다는 평가를 받은 반면, 언론・뉴스미디어가 잘 대응했다는 평가는 46%에 그쳐 6개 대응 주체 중 최하위를 나타냈다. 
같은 조사에서 정부는 67.2%, 대통령 및 청와대는 64.5%로 나타났다. 
또한 응답자의 56%는 국내 언론이 '검증되지 않은 정보와 가짜 뉴스를 많이 전달하고 있다'는 주장에 동의했다.
한국언론진흥재단 미디어연구센터가 3월 9일부터 12일까지 성인 1,000명을 대상 으로 실시한 온라인 설문조사에도 코로나19와 관련해 사회의 여러 주체가 적절히 대응 하고 있는지를 4점 척도로 물은 결과에서도 언론(63.7%)은 의료기관(93.2%), 지방자 치단체(77.2%), 정부(74.4%)보다 낮은 신뢰도를 나타냈다. 
확진자 또는 유증상자 (54.1%) 다음으로 신뢰하지 못하는 주체가 언론이었다.
코로나19 국면에서 언론의 이 같은 낮은 신뢰도와 관련, 이날 발제를 맡은 정준희 한양대 정보사회미디어학과 겸임교수는 '교란정보'라는 개념을 사용해 설명했다. 
정준희 교수는 "문제해결을 촉진시킨다는 면에서 미디어가 주는 공포의 순기능이 있지만 자극 의 정도를 높여가는 역기능도 만만치 않았다."고 지적하며 "'뚫렸다', '창궐', '대혼란' 같은 식의 과도한 기술을 통해 지속적으로 자극을 높여가며 각자가 이기적 선택을 하도 록 조장한 면이 있는지 반성해야 한다."고 했다. 
정 교수는 "뉴스가 감염병 국면에 도움이 못 되고 오히려 그릇된 감정을 전염시키는 경우가 있다."며 지금 같은 인포데믹(infodemic, 잘못된 정보가 전염병처럼 퍼지는 현상) 국면에선 치료적 보도가 필요하다 고 지적한 뒤 "악의적이고 의도적으로 유포되는 정보를 의미하는 '교란정보 (DISINFORMATION)'가 가장 큰 문제"라며 "교란정보에 맞춘 정보제공이 중요하고 이는 기성 언론을 통해 이뤄지는 것이 바람직하다."며 '교정정보'를 강조했다.
우리는 교란정보를 극복하고 있을까. 
정준희 교수는 "현재 해외의 상황 변화에 따라 교정정보가 상당 부분 유통됐다. 
해외 상황이 객관적으로 드러나면서 한국과 비교한 교정정보가 나타났다."고 지적했다. 
예컨대 일부 국내 언론이 지속적으로 방역 당국에 대한 '악의적' 보도를 이어갔으나, "바이러스가 시민적 자유를 시험하는 시대에, 도시를 계속 열어두면서 감염을 공격적으로 감시하는 이 전략이 먹히기만 한다면 민주사회에 본보기가 될 수 있다."(2/25)고 보도한 뉴욕타임스, "WHO(세계보건기구)는 한국이 코로나바이러스 감염 제한에 성공한 점을 들어 모든 나라에게 공격적인 테스크를 촉구 했다."(3/18)고 보도한 가디언 같은 외신보도가 국내에 확산되면서 국내 방역 당국의 수준이 객관적으로 드러났고, 이것이 '교정정보' 역할을 했다는 설명이다. 
정준희 교수는 "외신을 인용하는 언론 보도가 증가하며 팩트체크 노력이 이어졌고, 여기에 한국 방역 당국의 투명한 정보공개와 교정정보 확산 노력이 더해졌다."고 밝혔다.
이 같은 결과 방역 당국을 비판하는 언론보다 방역 당국을 신뢰하는 여론이 나타나고 있다는 지적이다. 
조슈아 벤튼 하버드대 니먼저널리즘랩 소장은 "어느 때보다 뉴스 이용자가 증가하고 있지만 언론사는 재정적인 어려움에 직면했다."고 지적했다. 
조슈아 벤튼 소장은 "코로나19는 현지의 지역 뉴스가 가장 중요하지만 대부분 재정적 어려움을 겪으며 문을 닫고 있다. 언론사들이 준비했던 각종 콘퍼런스는 취소되고 있고, 신문 배달도 위기 상황이다. 모든 매체의 광고매출이 감소하고 있고 언론사들은 비용축소와 통폐합이 예상된다."며 저널리즘의 붕괴를 우려했다.
이는 국내 언론이 처한 상황과도 유사하다. 
에이미 브리튼 워싱턴포스트 기자는 "이런 위기상황에는 검증 시간이 충분하지 않다. 대면 취재도 어려운 상황이다. 병원 간의 정보공유도 부족하다. 광고 수입도 급감했다. 그럼에도 불구하고 최근 워싱턴포스 트의 구독자가 어느 때보다 오르고 있다. 독자들은 진짜 정보를 원하고 있다."며 전 세계 기자들의 분투를 당부했다.
"""

byline = "정철운 기자"

In [ ]:
len(body)
바이라인 = "정철운 기자"
이미지개수 = 0
기사 길이 = 2251
평균 문장 길이 = 96.913
문장당 평균 부사 수 = 0
제목 길이 = 29
제목에 물음표 느낌표 수 = 0
제목의 부사 수 = 1
수치 인용 수 = 38
인용문 수 = 14
인용문 길이 비율 = 43.22

2274

In [ ]:
바이라인 = "정철운 기자"
이미지개수 = 0
기사 길이 = 2251
평균 문장 길이 = 96.913
문장당 평균 부사 수 = 0
제목 길이 = 29
제목에 물음표 느낌표 수 = 0
제목의 부사 수 = 1
수치 인용 수 = 38
인용문 수 = 14
인용문 길이 비율 = 43.22

In [ ]:
# 기사의 길이
# 기사 길이의 평균은 분류, 매체유형, 등에 따라 다름.
avg = 791.92
sd = 660.64
score_contentLength = 0

contentLength = 2274
if contentLength <= avg:
    score_contentLength = 0
if contentLength <= avg + 0.5 * sd:
    score_contentLength = 0.165
elif contentLength <= avg + sd:
    score_contentLength = 0.33
elif contentLength <= avg + 1.5 * sd:
    score_contentLength = 0.495
elif contentLength <= avg + 2.0 * sd:
    score_contentLength = 0.66
elif contentLength <= avg + 2.5 * sd:
    score_contentLength = 0.835
else:
    score_contentLength = 1

f"기사의 길이 점수:  {score_contentLength}"

'기사의 길이 점수:  0.835'

In [ ]:
# 인용문의 수
#stat = aggrStats.get_stat(category, mediaType, "quotes")

numQuotes = 14
if numQuotes < 15:
    score_quoteCount = numQuotes / 15
else:
    score_quoteCount = 1

f"인용문의 수 점수:  {score_quoteCount}"

'인용문의 수 점수:  0.9333333333333333'

In [ ]:
# 제목의 길이
#stat = aggrStats.get_stat(category, mediaType, "titleLength")

lenTitle = 29
if lenTitle <= 45:
    score_titleLength = 0
else:
    score_titleLength = -1

f"제목의 길이 점수:  {score_titleLength}"

'제목의 길이 점수:  0'

In [ ]:
# 제목에 물음표/느낌표
#stat = aggrStats.get_stat(category, mediaType, "titleNumPuncs")

numTitlePuncs = 0
if numTitlePuncs == 0:
    score_titlePuncCount = 0
elif numTitlePuncs == 1:
    score_titlePuncCount = -0.5
else:
    score_titlePuncCount = -1

f"제목의 물음표 느낌표 수 점수:  {score_titlePuncCount}"

'제목의 물음표 느낌표 수 점수:  0'

In [ ]:
# 수치 인용 수 
# 수치 인용 언론사마다 다르기때문에 평균과 표준편차가 있음.
#stat = aggrStats.get_stat(category, mediaType, "numNumber")
numNumbers = 38
if numNumbers < avg:
    score_numberCount = 0
elif numNumbers < (avg + 0.5 * sd):
    score_numberCount = 0.33
elif numNumbers < (avg + sd):
    score_numberCount = 0.66
else:
    score_numberCount = 1

f"수치 인용수 점수:  {score_numberCount}"

NameError: ignored

In [ ]:
# 이미지의 수
#stat = aggrStats.get_stat(category, mediaType, "imageCount")
imageCount = 0
if imageCount <= 0:
    score_imageCount = 0
elif imageCount == 1:
    score_imageCount = 0.33
elif imageCount == 2:
    score_imageCount = 0.66
elif imageCount == 3:
    score_imageCount = 1
elif imageCount == 4:
    score_imageCount = 0.66
elif imageCount == 5:
    score_imageCount = 0.33
else: #imageCount >= 6:
    score_imageCount = 0

f"이미지의 수 점수:  {score_imageCount}"

'이미지의 수 점수:  0'

In [ ]:
 # 평균 문장 길이
#stat = aggrStats.get_stat(category, mediaType, "avgSentenceLength")
avgSentenceLength = 0
if avgSentenceLength >= (stat.avg + stat.sd):
    score_avgSentenceLength = -1
else:
    score_avgSentenceLength = 0

f"평균 문장 길이 점수:  {score_avgSentenceLength}"

In [ ]:
# 제목의 부사수
#stat = aggrStats.get_stat(category, mediaType, "titleAdverbs")
numTitleAdverbs = 0
if numTitleAdverbs == 1:
    score_titleAdverbCount = -0.5
elif numTitleAdverbs >= 2:
    score_titleAdverbCount = -1
else:
    score_titleAdverbCount = 0

f"제목 부사수 점수:  {score_titleAdverbCount}"

In [ ]:
# 문장당 평균 부사수
#stat = aggrStats.get_stat(category, mediaType, "avgAdverbsPerSentence")
avgAdverbsPerSentence = 0
if avgAdverbsPerSentence >= (stat.avg + 2 * stat.sd):
    score_avgAdverbCountPerSentence = -1
else:
    score_avgAdverbCountPerSentence = 0

f"문장당 부사수 점수:  {score_avgAdverbCountPerSentence}"

In [ ]:
 # 인용문 길이 비율
#stat = aggrStats.get_stat(category, mediaType, "quotePercent")
quotePercent = 0
if quotePercent < 0.5:
    score_quotePercent = 0
elif quotePercent < 0.8:
    score_quotePercent = -0.5
else:
    score_quotePercent = -1

f"인용문 길이 비율 점수:  {score_quotePercent}"

In [ ]:
# 내가 할일 

# 1. 각각 언론사, 카테고리별 평균과 표준편차를 어떻게 구성해놓았는지 수치를 찾을것
# 2. 위의 기사를 전처리하여 언론사 평균 표준편차를 적용하고 부사수 등을 뽑아 처리해서 비슷한 결과가 나오는지 테스트.
# 3. 도출된 점수들을 기반으로 총점을 구하고 
# 4. 각각의 저널리즘 가치 점수를 계산해볼것 
# 독이성 투명성 사실성 유용성 균형성 다양성 독창성 중요성 심층성 선정성
# 저널리점 가치부여의 한계점 정리. 그리고 각각 점수를 측정하는 부분의 한계점도 정리할것.

In [ ]:
# 독이성
journal_read = score_byline * 0.001 \
                + score_contentLength * 0.003 \
                + score_quoteCount * 0.001 \
                + score_titleLength * 1 \
                + score_titlePuncCount * 1.002 \
                + score_numberCount * 1.354 \
                + score_imageCount * 1.5 \
                + score_avgSentenceLength * 1.5 \
                + score_titleAdverbCount * 2.466 \
                + score_avgAdverbCountPerSentence * 0.5
# 투명성
journal_clear = score_byline * 4.498 \
                + score_contentLength * 3.003 \
                + score_quoteCount * 4.5 \
                + score_titlePuncCount * 3.619 \
                + score_numberCount * 1.454 \
                + score_imageCount * 1 \
                + score_quotePercent * 0.001
# 사실성
journal_truth = score_byline * 4.493 \
                + score_contentLength * 3.503 \
                + score_quoteCount * 3.501 \
                + score_titlePuncCount * 0.001 \
                + score_numberCount * 0.502 \
                + score_imageCount * 1.5 \
                + score_titleAdverbCount * 0.5 \
                + score_avgAdverbCountPerSentence * 1.5 \
                + score_quotePercent * 1
# 유용성
journal_useful = score_byline * 3.494 \
                    + score_contentLength * 3.498 \
                    + score_quoteCount * 2.001 \
                    + score_numberCount * 1.956 \
                    + score_imageCount * 1
# 균형성
journal_balance = score_byline * 2.996 \
                    + score_contentLength * 3.002 \
                    + score_quoteCount * 3 \
                    + score_titlePuncCount * 1.501 \
                    + score_titleAdverbCount * 0.501 \
                    + score_avgAdverbCountPerSentence * 1 \
                    + score_quotePercent * 1 \
# 다양성
journal_variety = score_byline * 0.998 \
                    + score_contentLength * 4.994 \
                    + score_quoteCount * 2.501 \
                    + score_titleLength * 0.5 \
                    + score_numberCount * 1.953 \
                    + score_imageCount * 1 \
                    + score_avgSentenceLength * 0.5 \
                    + score_quotePercent * 0.5
# 독창성
journal_original = score_byline * 4.494 \
                    + score_contentLength * 4.492 \
                    + score_quoteCount * 3.501 \
                    + score_titlePuncCount * 3.09 \
                    + score_numberCount * 1.823 \
                    + score_imageCount * 1.501
# 중요성
journal_important = score_byline * 2.495 \
                    + score_contentLength * 3.503 \
                    + score_quoteCount * 3.5 \
                    + score_numberCount * 1.002 \
                    + score_imageCount * 0.5
# 심층성
journal_deep = score_byline * 4.496 \
                + score_contentLength * 4.995 \
                + score_quoteCount * 3.501 \
                + score_numberCount * 1.336 \
                + score_imageCount * 1 \
                + score_quotePercent * 1
# 선정성
journal_yellow = score_byline * 4.491 \
                    + score_titleLength * 3.5 \
                    + score_titlePuncCount * 3.501 \
                    + score_titleAdverbCount * 3.5 \
                    + score_avgAdverbCountPerSentence * 3.5 \
                    + score_quotePercent * 3.5

ret.journalSum = journal_read + journal_clear + journal_truth + journal_useful + journal_balance \
                    + journal_variety + journal_original + journal_important + journal_deep + journal_yellow